# Plotting API
---

UXarray provides a feature-rich plotting API for natively visualuzing unstructured grids, with or without data variables.

This notebook introduces how to interface with the plotting methods through UXarray's core data structures and provides an introduction to methods that are covered in detail in the next sections.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">See also:</p>
    This notebook acts as an introduction into using the UXarray plotting API. Please refer to the following notebooks in this chapter for a detailed
    overview of visualization techniques for different purposes (e.g. <a href="https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/03-uxarray-vis/02-grid-topology.html">Grid Topology</a>, <a href="https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/03-uxarray-vis/03-polygons.html">Polygons</a>
</div>

## UXarray Plotting API Design

Before jumping into any code, let's take a look at a high-level snapshot of UXarray's API design from a Unifed Modeling Language (UML)-like standpoint.


<!-- Using text-align on a parent div -->
<div style="text-align: center;">
    <img src="../images/plotting_api/uxarray-plot-api-design.png" alt="UXarray Plotting API Design" width="1000"/>
</div>



Key takeaways from this design are that:

- UXarray's unified grid representation (through the UGRID conventions) means that all visualization functionality is agnostic to the grid format initially provided by the user.
- Each Uxarray data structure (i.e. `Grid`, `UxDataset`, `UxDataArray`) has its own `.plot` accessor, which is used to call plotting routines.
- The visualization functionality through these `.plot` accessors use HoloViz packages' plotting functions, wrapping them in a way to exploit all the information that comes from unstructured grids (e.g. connectivity) and provide our unstructured grids-specific functions in the most convenient way for the user.
- `Grid` additionally provides conversion functions that generate `SpatialPandas.GeoDataFrame` as well as `Matplotlib.PolyCollection` and `Matplotlib.LineCollection` data structures to be visualized in HoloViz packages and Matplotlib, respectively, at the user's own convenience.

Now, we can see the API in action on a sample dataset.

## Imports

In [ ]:
import uxarray as ux

In [ ]:
base_path = "../../meshfiles/"
grid_filename = base_path + "oQU480.grid.nc"
data_filename = base_path + "oQU480.data.nc"

In [ ]:
grid = ux.open_grid(grid_filename)
uxds = ux.open_dataset(grid_filename, data_filename)

## Grid

Since a `Grid` instance only contains information about the topology of an unstructured grid (a.k.a. no data variables), the visualizations generated from the `Grid` class only showcase the coordinates and connectivity.

By default, the `Grid.plot` method renders the borders of each of the faces.

In [ ]:
grid.plot(title="Default Grid Plot")

The UXarray plotting API is written with HoloViews, with the default backend used for generating plots being `bokeh`. This means that by default, all plots are enabled with interactive features such as panning and zooming. In addition to `bokeh`, UXarray also supports the `matplotlib` backend.

For the remainder of this notebook, we will use the `matplotlib` backend to generate static plots.

ADD ADMONITION LEADING USERS TO THE CUSTOMIZATION SECTION!


In [ ]:
grid.plot(
    title="Default Grid Plot with Matplotlib",
    backend="matplotlib",
    aspect=2,
    fig_size=500,
)



You can call specific plotting routines through the `plot` accessor

In [ ]:
grid.plot.nodes(title="Grid Node Plot", backend="matplotlib", aspect=2, fig_size=500)

Since each `UxDataset` and `UxDataArray` is always linked to a `Grid` instance through the `uxgrid` attribute, all of these grid-specific visualizations are accessible by using that attribute.


In [ ]:
uxds.uxgrid.plot(
    title="Grid plot through uxgrid attribute",
    backend="matplotlib",
    aspect=2,
    fig_size=500,
)

## UxDataArray Plotting

The default plotting method is a great starting point for visualizations. It selects what visualization method to use based on the grid element that the data is mapped to (nodes, edges, faces) and the number of elements in the mesh.

**Face-Centered Variable:**
* Less than 10,000 Faces: Vector Polygons, including polygons split across the antimeridian.
* More than 10,000 Faces: Pasteurized Polygons, excluding polygons crossing the antimeridian.

**Edge-Centered Variable:**
* Edge Center Coordinates (i.e. `edge_lon`, `edge_lat`) plotted as Points, shaded with the data variable

**Node-Centered Variable:**
* Node Center Coordinates (i.e. `node_lon`, `node_lat`) plotted as Points, shaded with the data variable.

In our dataset, `bottomDepth` is a face-centered variable and our grid has less than 10,000 faces, meaning that the resulting plot should be a vector plot with polygons split across the antimeridian.


In [ ]:
uxds["bottomDepth"].plot(
    title="Default UxDataArray Plot", backend="matplotlib", aspect=2, fig_size=500
)

We can also call other plotting methods through the `plot` accessor, as was the case with the `Grid` class.

For example, if we wanted to rasterize the polygons and exclude the ones that cross the antimeridian, it would look something like the following.

In [ ]:
uxds["bottomDepth"].plot.rasterize(
    method="polygon",
    exclude_antimeridian=True,
    title="Default UxDataArray Plot",
    backend="matplotlib",
    aspect=2,
    fig_size=500,
)

In addition to the grid-aware plotting functionality provided by UXarray, all the existing Xarray functionality is still avaliable, albeit with a high level of discretion, since not all methods are guaranteed to work on unstructured grids 

In [ ]:
# uxds["bottomDepth"].plot.histogram()

## `UxDataset` Plotting

As of the most recent release, there are no UXarray specific plotting routines that can be called through a `UxDataset`. However, existing Xarray functionality is still available. 

TODO!!!!

In [ ]:
uxds.plot()